<a href="https://colab.research.google.com/github/ryann-arruda/deep_learning_algorithms/blob/main/perceptron_step_activation_function_and_minibatch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection  import train_test_split
import pandas as pd
import torch

## **Loading Dataset**

In [2]:
data = load_breast_cancer()

In [3]:
x = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.DataFrame(data.target, columns=['Target'])

## **Separating the Dataset**

In [4]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=27)

## **Converting Data to Tensors**

In [5]:
x_train = torch.tensor(x_train.values, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32)

In [6]:
x_test = torch.tensor(x_test.values, dtype=torch.float32)
y_test = torch.tensor(y_test.values, dtype=torch.float32)

## **Perceptron**

In [34]:
class Perceptron():
  def __init__(self, num_features, lr=1.0):
    self.num_featues = num_features
    self.weights = torch.zeros(num_features)
    self.bias = torch.tensor(0.)
    self.lr = torch.tensor(lr)

  def forward(self, x):
    z = self.weights.matmul(x.T) + self.bias

    return torch.where(z >= 0, 1., 0.)

  def fit(self):
    pass